# Lesson 7 - 2021/11/25

## Pandas

In [1]:
import pandas as pd

map_dict = {'ENST': 'RNA', 'ENSG': 'gene', 'ENSP': 'protein'}
count_dict = {'ENST': 3300, 'ENSG': 18435, 'ENSP': 12034}
groups_dict = {'ENST': 13, 'ENSG': 42, 'ENSP': 157}
 
df = pd.DataFrame({'mapping type': map_dict, 'counts': count_dict, 'classes': groups_dict})
df

,mapping type,counts,classes
ENST,RNA,3300,13
ENSG,gene,18435,42
ENSP,protein,12034,157


In [2]:
df['mapping type']

ENST        RNA
ENSG       gene
ENSP    protein
Name: mapping type, dtype: object

Adding a new column using the dictionary-style syntax:

In [3]:
df['averages'] = df['counts'] / df['classes']

df

,mapping type,counts,classes,averages
ENST,RNA,3300,13,253.846154
ENSG,gene,18435,42,438.928571
ENSP,protein,12034,157,76.649682


### Access to data


In [4]:
df.counts

ENST     3300
ENSG    18435
ENSP    12034
Name: counts, dtype: int64

In [5]:
df['counts']

ENST     3300
ENSG    18435
ENSP    12034
Name: counts, dtype: int64

In [6]:
df['ENST':'ENSG']

,mapping type,counts,classes,averages
ENST,RNA,3300,13,253.846154
ENSG,gene,18435,42,438.928571


For `Series` with explicit integer indexes

In [7]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])

data

1    a
3    b
5    c
dtype: object

indexing operations will use the explicit indices

In [8]:
data[1]   # explicit index when indexing

'a'

while slicing operations will use the implicit Python-style indexes:

In [9]:
data[1:3] # implicit index when slicing

3    b
5    c
dtype: object

The ``loc`` attribute allows indexing and slicing that always references the explicit index.

In [10]:
data.loc[1]

'a'

In [11]:
data.loc[1:3]

1    a
3    b
dtype: object

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index.

In [12]:
data.iloc[1]

'b'

In [13]:
data.iloc[1:3]

3    b
5    c
dtype: object

#### DataFrame as two-dimensional array

In [14]:
df.values

array([['RNA', 3300, 13, 253.84615384615384],
       ['gene', 18435, 42, 438.92857142857144],
       ['protein', 12034, 157, 76.64968152866243]], dtype=object)

Using the `iloc` and `loc` indexers, we can index the underlying array

In [15]:
df

,mapping type,counts,classes,averages
ENST,RNA,3300,13,253.846154
ENSG,gene,18435,42,438.928571
ENSP,protein,12034,157,76.649682


In [16]:
df.iloc[1:, 1:]

,counts,classes,averages
ENSG,18435,42,438.928571
ENSP,12034,157,76.649682


In [17]:
df.loc['ENSG':, 'counts':]

,counts,classes,averages
ENSG,18435,42,438.928571
ENSP,12034,157,76.649682


##### Boolean indexing

In [18]:
df['classes']

ENST     13
ENSG     42
ENSP    157
Name: classes, dtype: int64

In [19]:
df[df.counts > 5000]

,mapping type,counts,classes,averages
ENSG,gene,18435,42,438.928571
ENSP,protein,12034,157,76.649682


## Handling Missing Data

To indicate the presence of missing data in a table, Pandas uses two Python null values: the `None` object, and the special floating-point `NaN` value.

#### `None`: Pythonic missing data

In [20]:
import numpy as np

vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Aggregate operations like `sum()` across an array with a ``None`` value leads to errors.

In [21]:
sum(vals1)

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### `NaN`: Missing numerical data

`NaN` (*Not a Number*) is a special floating-point value.

In [22]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [23]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide special aggregations that ignore the missing values:

In [24]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## Operating on Null Values

In [25]:
df.loc['ENSG', 'counts'] = np.nan
df.loc['ENSG', 'classes'] = np.nan
df.loc['ENST', 'classes'] = np.nan
df

,mapping type,counts,classes,averages
ENST,RNA,3300.0,NaN,253.846154
ENSG,gene,NaN,NaN,438.928571
ENSP,protein,12034.0,157.0,76.649682


### Dropping null values

By default, `dropna()` drops all rows in which *any* null value is present.

In [26]:
df.dropna() # It returns the modified Dataframe without the 'ENST' and 'ENSG' rows

,mapping type,counts,classes,averages
ENSP,protein,12034.0,157.0,76.649682


You can drop NA values along a different `axis`: indeed:

In [27]:
df.dropna(axis=1)  # It returns the modified Dataframe without the 'counts' and 'classes' columns

,mapping type,averages
ENST,RNA,253.846154
ENSG,gene,438.928571
ENSP,protein,76.649682


In [28]:
df.dropna(axis='columns')

,mapping type,averages
ENST,RNA,253.846154
ENSG,gene,438.928571
ENSP,protein,76.649682


In [29]:
df.dropna(axis=1, how='all') 

,mapping type,counts,classes,averages
ENST,RNA,3300.0,NaN,253.846154
ENSG,gene,NaN,NaN,438.928571
ENSP,protein,12034.0,157.0,76.649682


The ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [30]:
df.dropna(axis=1, thresh=2)  # The 'classes' column is dropped because it doensn't have at least 2 non-nul values

,mapping type,counts,averages
ENST,RNA,3300.0,253.846154
ENSG,gene,NaN,438.928571
ENSP,protein,12034.0,76.649682


###  Filling null values

We can replace missing data with:

- a constant value

In [31]:
df.fillna(0) # <=> df.replace(np.nan, 0) # The methods return the modified Dataframe

,mapping type,counts,classes,averages
ENST,RNA,3300.0,0.0,253.846154
ENSG,gene,0.0,0.0,438.928571
ENSP,protein,12034.0,157.0,76.649682


- a mean, median or mode of the column to the missing data belongs

In [32]:
df.fillna(df.mean()) # It returns the modified Dataframe

,mapping type,counts,classes,averages
ENST,RNA,3300.0,157.0,253.846154
ENSG,gene,7667.0,157.0,438.928571
ENSP,protein,12034.0,157.0,76.649682


## Load Data from CSV Files

In [33]:
!head ../data/brca_transcripts.txt

transcript_id	biotype	bp	aa
ENST00000352993.7	Protein coding	3668	721
ENST00000354071.7	Protein coding	4497	1399
ENST00000461221.5	Nonsense mediated decay	5693	63
ENST00000461574.1	Protein coding	726	242
ENST00000461798.5	Nonsense mediated decay	582	63


This type of files can be load into a Pandas `DataFrame` using the `read_csv` function in Pandas:

In [34]:
brca1_df = pd.read_csv('../data/brca_transcripts.txt', sep = '\t', index_col = 0, header = 0)

brca1_df

,biotype,bp,aa
transcript_id,,,
ENST00000352993.7,Protein coding,3668,721
ENST00000354071.7,Protein coding,4497,1399
ENST00000461221.5,Nonsense mediated decay,5693,63
ENST00000461574.1,Protein coding,726,242
ENST00000461798.5,Nonsense mediated decay,582,63


## Aggregation and Grouping

In [35]:
brca1_df.describe()

,bp,aa
count,5.000000,5.00000
mean,3033.200000,497.60000
std,2288.655216,571.29572
min,582.000000,63.00000
25%,726.000000,63.00000
50%,3668.000000,242.00000
75%,4497.000000,721.00000
max,5693.000000,1399.00000


We can compute statistics conditionally on some label or index:

In [36]:
type(brca1_df.groupby('biotype'))

pandas.core.groupby.generic.DataFrameGroupBy

In [37]:
brca1_df.groupby('biotype').describe()

bp                                            \
                        count         mean          std    min      25%   
biotype                                                                   
Nonsense mediated decay   2.0  3137.500000  3614.022759  582.0  1859.75   
Protein coding            3.0  2963.666667  1981.709952  726.0  2197.00   

                                                    aa              \
                            50%      75%     max count        mean   
biotype                                                              
Nonsense mediated decay  3137.5  4415.25  5693.0   2.0   63.000000   
Protein coding           3668.0  4082.50  4497.0   3.0  787.333333   

                                                                          
                                std    min    25%    50%     75%     max  
biotype                                                                   
Nonsense mediated decay    0.000000   63.0   63.0   63.0    63.0    63.0  
Protein coding           581.345279  242.0  481.5  721.0  1060.0  1399.0

In [38]:
brca1_df.groupby('biotype')['bp'].mean()

biotype
Nonsense mediated decay    3137.500000
Protein coding             2963.666667
Name: bp, dtype: float64

### apply
The ``apply()`` method lets you apply a function to the group results.

In [39]:
brca1_df.groupby('biotype')[['bp', 'aa']].apply(np.sum)

,bp,aa
biotype,,
Nonsense mediated decay,6275,126
Protein coding,8891,2362


In [40]:
brca1_df[['bp', 'aa']].apply(np.sum)            # Total nucleotides and total aminoacids

bp    15166
aa     2488
dtype: int64

In [41]:
brca1_df[['bp', 'aa']].apply(np.sum, axis=1)    # Nucleotides + aminoacids for each transcript

transcript_id
ENST00000352993.7    4389
ENST00000354071.7    5896
ENST00000461221.5    5756
ENST00000461574.1     968
ENST00000461798.5     645
dtype: int64

We can also define an arbitrary function:

In [42]:
def function(row, value):
    status = ''
    if row['bp'] >= value:
        status = 'High'
    else:
        status = 'Low'
        
    return status

In [43]:
bp_mean = brca1_df['bp'].mean()
print('bp mean:', bp_mean)

bp mean: 3033.2


In [44]:
type(brca1_df.apply(function, args=(bp_mean,), axis=1))

pandas.core.series.Series

In [45]:
brca1_df['transcript_length'] = brca1_df.apply(
    function,
    args=(bp_mean,),
    axis=1
)

brca1_df

,biotype,bp,aa,transcript_length
transcript_id,,,,
ENST00000352993.7,Protein coding,3668,721,High
ENST00000354071.7,Protein coding,4497,1399,High
ENST00000461221.5,Nonsense mediated decay,5693,63,High
ENST00000461574.1,Protein coding,726,242,Low
ENST00000461798.5,Nonsense mediated decay,582,63,Low


### Lambda function

Python <strong>lambdas</strong> are anonymous functions, subject to a more restrictive but more concise syntax than regular Python functions.

In [46]:
brca1_df['protein_length'] = brca1_df.apply(
    lambda row, value: 'High' if row['bp'] > value else 'Low',
    args = (brca1_df['bp'].mean(),),
    axis = 1
)

brca1_df

,biotype,bp,aa,transcript_length,protein_length
transcript_id,,,,,
ENST00000352993.7,Protein coding,3668,721,High,High
ENST00000354071.7,Protein coding,4497,1399,High,High
ENST00000461221.5,Nonsense mediated decay,5693,63,High,High
ENST00000461574.1,Protein coding,726,242,Low,Low
ENST00000461798.5,Nonsense mediated decay,582,63,Low,Low
